<a href="https://colab.research.google.com/github/fpetitit/hackathon-alimentation/blob/main/Hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Fruits_Légumes_Kantar.csv' , encoding='latin-1', sep = ';' )

<ipython-input-292-14f127147c82>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/Fruits_Légumes_Kantar.csv' , encoding='latin-1', sep = ';' )


In [ ]:
df.head()

,duree,annee,periode,geog,Libellé_Court,Q_ach,S_dep,nbr_act,penet,nbr_ach
0,A,2005,A,200 000 HABITANTS ET PLUS,6 FRUITS,2.589160e+08,4.141993e+08,1.789321e+08,96.6,5635084.671
1,A,2005,A,200 000 HABITANTS ET PLUS,6 FRUITS + 9 LEGUMES_BIO,1.191566e+07,2.473982e+07,7.607372e+06,28.7,1672721.256
2,A,2005,A,200 000 HABITANTS ET PLUS,6 FRUITS + 9 LEGUMES_NON BIO,5.830080e+08,9.541179e+08,2.907253e+08,99.0,5776878.199
3,A,2005,A,200 000 HABITANTS ET PLUS,6 FRUITS hors BANANE + 9 LEGUMES_BIO,1.191566e+07,2.473982e+07,7.607372e+06,28.7,1672721.256
4,A,2005,A,200 000 HABITANTS ET PLUS,6 FRUITS hors BANANE + 9 LEGUMES_NON BIO,5.268689e+08,8.690435e+08,2.735817e+08,99.0,5776399.640


In [ ]:
df = df.rename(columns={'Libellé_Court': 'label', 'Q_ach' : 'quantite'})
df = df[df.annee >= 2013]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2062880 entries, 187655 to 2679622
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   duree     object 
 1   annee     int64  
 2   periode   object 
 3   geog      object 
 4   label     object 
 5   quantite  float64
 6   S_dep     float64
 7   nbr_act   float64
 8   penet     float64
 9   nbr_ach   float64
dtypes: float64(5), int64(1), object(4)
memory usage: 173.1+ MB


In [ ]:
df.shape

(2062880, 10)

In [ ]:
def find_tomate(df, colonne='label'):
  regex = r'\btomate(?:s?|_[A-Za-z0-9]+)?\b'
  resultats = df[df[colonne].str.contains(regex, flags=re.IGNORECASE, na=False)]
  return resultats

def nettoyer_libelle(df, colonne='label', remplace_underscore=True, supprime_non_bio=True, remplace_espaces=True, supprime_espace_fin=True):
    if remplace_underscore:
        df.loc[:, colonne] = df[colonne].str.replace('_', ' ', regex=False)

    if supprime_non_bio:
        df.loc[:, colonne] = df[colonne].str.replace('NON BIO', '', regex=False)

    if remplace_espaces:
        df.loc[:, colonne] = df[colonne].str.replace(r'\s{2,}', ' ', regex=True)

    if supprime_espace_fin:
        df.loc[:, colonne] = df[colonne].str.rstrip()

    return df

def separer_tomates(df, colonne='label'):
    df['BIO'] = df[colonne].apply(lambda x: 'BIO' if 'BIO' in str(x).upper() else 'NON BIO')
    #df.loc[:,colonne] = df[:,colonne].apply(lambda x: True if 'BIO' in str(x).upper() else False)
    #df_bio = df[df[colonne].str.contains('BIO', flags=re.IGNORECASE, na=False)]
    #df_non_bio = df[~df[colonne].str.contains('BIO', flags=re.IGNORECASE, na=False)]
    return df #, df_bio, df_non_bio

In [ ]:
df_tomate = find_tomate(df)
df_tomate = nettoyer_libelle(df_tomate)
test = separer_tomates(df_tomate, colonne='label')
#test, df_bio, df_non_bio = separer_tomates(df_tomate, colonne='label')

<ipython-input-304-b9d8eb288cb2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BIO'] = df[colonne].apply(lambda x: 'BIO' if 'BIO' in str(x).upper() else 'NON BIO')


In [ ]:
test.head()

,duree,annee,periode,geog,label,quantite,S_dep,nbr_act,penet,nbr_ach,BIO
187980,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE,1.009684e+08,2.342191e+08,1.191554e+08,95.8,6.128174e+06,NON BIO
187981,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE ALLONGEE,9.010526e+06,2.212908e+07,1.372715e+07,58.2,3.720937e+06,NON BIO
187984,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE AUTRE ORIGINE,1.475572e+06,2.813036e+06,1.690117e+06,11.6,7.445315e+05,NON BIO
187985,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE AUTRES,9.055938e+06,3.068747e+07,1.418191e+07,60.1,3.842228e+06,NON BIO
187988,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE BELGIQUE,2.880580e+06,4.921111e+06,3.169431e+06,18.3,1.173218e+06,NON BIO


In [ ]:
import pandas as pd
import plotly.express as px

def graphique_quantite_par_annee(df, colonne_label='label', colonne_quantite='quantité', colonne_annee='année'):

    df_grouped = df.groupby([colonne_annee, 'BIO'])[colonne_quantite].sum().reset_index()

    fig = px.line(df_grouped, x=colonne_annee, y=colonne_quantite, color='BIO', markers=True,
                  title="Quantité consommée par année (Bio vs Non Bio)",
                  labels={colonne_annee: 'Année', colonne_quantite: 'Quantité', 'BIO': 'Type de tomate'})

    fig.show()

In [ ]:
graphique_quantite_par_annee(test, 'label', 'quantite', 'annee')

In [ ]:
test_bio = test[test.BIO == 'BIO']
df_grouped = test_bio.groupby('annee')['quantite'].sum().reset_index()

fig = px.line(df_grouped, x='annee', y='quantite', markers=True,
                  title="Quantité consommée par année (Bio vs Non Bio)",
                  labels={'annee': 'Année', 'quantite': 'Quantité', 'BIO': 'Type de tomate'})

fig.show()

In [ ]:
test_bio.head()

,duree,annee,periode,geog,label,quantite,S_dep,nbr_act,penet,nbr_ach,BIO
187989,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE BIO,4596616.821,1.415124e+07,5.310808e+06,20.8,1.329858e+06,BIO
188346,A,2013,A,65 ANS ET PLUS,TOMATE BIO,3993260.317,1.222478e+07,5.088858e+06,21.4,1.488490e+06,BIO
188703,A,2013,A,AGGLOMERATION PARISIENNE,TOMATE BIO,2222052.418,8.677805e+06,3.493294e+06,22.2,1.023196e+06,BIO
189263,A,2013,A,AISEE,TOMATE BIO,3340746.065,1.157131e+07,4.031823e+06,23.9,9.732950e+05,BIO
189584,A,2013,A,ARTISAN COMMERC CHEF ENTR,TOMATE BIO,488972.667,1.567247e+06,5.398863e+05,14.2,1.600654e+05,BIO


In [ ]:
test.head()

,duree,annee,periode,geog,label,quantite,S_dep,nbr_act,penet,nbr_ach,BIO
187980,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE,1.009684e+08,2.342191e+08,1.191554e+08,95.8,6.128174e+06,NON BIO
187981,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE ALLONGEE,9.010526e+06,2.212908e+07,1.372715e+07,58.2,3.720937e+06,NON BIO
187984,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE AUTRE ORIGINE,1.475572e+06,2.813036e+06,1.690117e+06,11.6,7.445315e+05,NON BIO
187985,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE AUTRES,9.055938e+06,3.068747e+07,1.418191e+07,60.1,3.842228e+06,NON BIO
187988,A,2013,A,200 000 HABITANTS ET PLUS,TOMATE BELGIQUE,2.880580e+06,4.921111e+06,3.169431e+06,18.3,1.173218e+06,NON BIO


In [ ]:
test_bio = test.loc[test['BIO'] == 'BIO']


In [ ]:
test_bio = test_bio[test_bio['quantite'] > 0

In [ ]:


# Exemple de DataFrame fictif
data = {
    'label': ['TOMATE_BIO', 'TOMATE_NON BIO', 'TOMATE_BIO', 'TOMATE_NON BIO', 'TOMATE_BIO'],
    'quantité': [100, 150, 200, 250, 300],
    'date': ['2021-05-01', '2021-06-01', '2022-01-01', '2022-02-01', '2022-05-01']
}

val = pd.DataFrame(data)

# Appliquer la fonction
graphique_quantite_par_annee(df)
